In [ ]:
import numpy as np

<h3>Setting up the parameters

In [ ]:
dE=1E5                         #total changes between two iterations
thresh = 1E-3                  #dE tresh of change
discount = 0.9                 #discount factor
max_itera = 1E2                #before exiting the loop

x1_size = 20                   #the range is (2, 20)
x2_size = 20                   #the range is (2, 20)
nb_obstacles = 200             #the maximum of obstacles is (x1_size * x2_size) - (x1_size + x2_size ) + 1 in theory but might be lower if the first obstacles create a turn, which will make impossible to find a way from start to end
nb_checkpoints = 100           #the maximum of checkpoints is (x1_size * x2_size) -2, all the case except the end and start
rewards = [1000, 0.01, -0.02]    #rewards for [end, checkpoints, else]

#Note that as the grid_size goes up (same with the number of checkpoints), the end reward has to go up (because the big reward is more far away)

<h3>Reward, state and actions functions 

In [ ]:
#reward function for the state-value
def R(s, checkpoints, end_state, rewards):
    if s == end_state: return rewards[0]
    elif s in checkpoints: return rewards[1]
    else: return rewards [2]
    
#get the next possibles states and actions of the current state
def getNextStatesAndNextActions(state, actions, states):
    next_states = []
    next_actions = []
    for a in actions:
        next_state = (state[0] + actions[a][0], state[1] + actions[a][1])
        if next_state in states:
            next_states.append(next_state)
            next_actions.append(a)
    return next_states, next_actions

#when going in a direction, the robot also check the right and left state-value
def getLeftRightNextStates(state, next_state, states):
    direction = (next_state[0] - state[0], next_state[1] - state[1])
    left_offset = (direction[1], -direction[0])  # Rotate 90° counterclockwise
    right_offset = (-direction[1], direction[0]) # Rotate 90° clockwise

    left_state = (state[0] + left_offset[0], state[1] + left_offset[1])
    right_state = (state[0] + right_offset[0], state[1] + right_offset[1])

    left_state = left_state if left_state in states else state  # Ensure valid states
    right_state = right_state if right_state in states else state

    return left_state, right_state

<h3>Creating the grid and ensure it can be solved

In [ ]:
#Using BFS, ensure that a way is possible from the start to the end
def isReachable(start_state, end_state, states):
    queue = [start_state]
    visited = set()
    
    while len(queue)>0:
        x, y = queue.pop(0)
        
        if (x,y) == end_state:
            return True
        
        if (x,y) in visited:
            continue
        visited.add((x,y))
        
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            new_x, new_y = x + dx, y + dy
            if((new_x, new_y) in states and (new_x, new_y) not in visited):
                queue.append((new_x, new_y))    
    return False
    

#creating the grid with the obstacles and checkpoints
def createGrid(x1_size, x2_size, nb_obstacles, nb_checkpoints):
    states = set((i,j) for i in range(x1_size) for j in range(x2_size))
    end_state = (x1_size-1, x2_size-1)
    start_state = (0,0)
    obstacles = set()
    checkpoints = set()
    
    #creating the obstacles
    while(len(obstacles)<nb_obstacles):
        rand_tuple = (np.random.randint(0, x1_size), np.random.randint(0, x2_size))
        if rand_tuple not in obstacles and rand_tuple != end_state and rand_tuple != start_state:
            obstacles.add(rand_tuple)
            states.remove(rand_tuple)
            if not isReachable(start_state, end_state, states):
                obstacles.remove(rand_tuple)
                states.add(rand_tuple)
    
    #creating checkpoints
    while(len(checkpoints)<nb_checkpoints):
        rand_tuple = (np.random.randint(0, x1_size), np.random.randint(0, x2_size))
        if rand_tuple not in checkpoints and rand_tuple not in obstacles and rand_tuple != end_state and rand_tuple != start_state:
            checkpoints.add(rand_tuple)
    
    print('States:', list(states))
    print('Obstacles:', list(obstacles))
    print('Checkpoints:',list(checkpoints))
    print('End state:', end_state)
    
    return list(states), end_state, list(checkpoints), list(obstacles)

<h3>Setting up the differents states and initializing the state-value and policy functions

In [ ]:
states, end_state, checkpoints, obstacles = createGrid(x1_size, x2_size, nb_obstacles, nb_checkpoints)
actions = {'N':(0,1), 'E':(1,0), 'S': (0,-1), 'W': (-1,0)}

V = {s: 0 for s in states}
P = {s: None for s in states}

<h3>Using the value iteration algorithm to determine the state-value and policy of each state

In [ ]:
itera = 0
while(dE > thresh and itera < max_itera):
    dE = 0
    V_ = V.copy() #to not modify the state-value during during the iteration
    for s in states:
        if s == end_state: #no state-value for the end state
            continue
        next_states, next_actions = getNextStatesAndNextActions(s, actions, states)
        tmp = []
        for ns in next_states:
            left_state, right_state = getLeftRightNextStates(s, ns, states)
            tmp.append(R(ns, checkpoints, end_state, rewards) + discount * (0.8 * V[ns] + 0.1 * V[left_state] + 0.1 * V[right_state])) #storing each next state state-value
            
        #if a state is surrounded (no possible next states), it becomes an obstacle
        if(len(tmp)==0):
            V_[s] = V[s]
            P[s] = ' '
            states.remove(s)
            obstacles.append(s)
            print("Case",s,'surrounded by obstacles -> it becomes an obstacle' )
            continue
        
        V_[s] = np.max(tmp)                  #choosing the next state that maximize the state-value
        P[s] = next_actions[np.argmax(tmp)]  #storing the state to next state action
        dE = max(dE, abs(V_[s] - V[s]))      #storing the maximum change during the iteration
    V = V_                                 #applying changes
    itera+=1

<h3> Console print of state_values and policies

In [ ]:
print('Number of iterations = ',itera)

#not nice, I apologize
print("Optimal Value Function:")
for y in reversed(range(0, x2_size)):
    print([round(V.get((x, y), 0), 2) for x in range(0, x1_size)])

print("\nOptimal Policy:")
for y in reversed(range(0, x2_size)):
    print([P.get((x, y), ' ') for x in range(0, x1_size)])

<h3> Graphic visualization of policies, with the one used from the start to the end

In [ ]:
import pygame

#Parameters
grid_size = (x1_size, x2_size)
max_size = 800
cell_size = min(max_size // grid_size[0], max_size // grid_size[1])
width, height = grid_size[0] * cell_size, grid_size[1] * cell_size

#Initialization
pygame.init()
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Grid visualization")

#Grid display
def draw_grid():
    for x in range(grid_size[0]):
        for y in range(grid_size[1]):
            rect = pygame.Rect(x * cell_size, (grid_size[1] - 1 - y) * cell_size, cell_size, cell_size)
            color = (0, 255, 0) if (x, y) in checkpoints else (255, 255, 255)                            #checkpoints in green, obstacles in black, available cases in white
            if (x, y) in obstacles:
                color = (0, 0, 0)
            pygame.draw.rect(screen, color, rect)
            pygame.draw.rect(screen, (0, 0, 0), rect, 1)
            
            if (x, y) in P:
                draw_arrow(x, y, P[(x, y)], (0, 0, 0))
    
    font = pygame.font.Font(None, int(cell_size / 1.8))
    end_text = font.render("End", True, (255, 0, 0))
    screen.blit(end_text, ((grid_size[0] - 1) * cell_size + 5, 5))

#Arrows display
def draw_arrow(x, y, direction, color):
    center = (x * cell_size + cell_size // 2, (grid_size[1] - 1 - y) * cell_size + cell_size // 2)
    if direction == "N":
        pygame.draw.polygon(screen, color, [(center[0], center[1] - 10), (center[0] - 10, center[1] + 10), (center[0] + 10, center[1] + 10)])
    elif direction == "S":
        pygame.draw.polygon(screen, color, [(center[0], center[1] + 10), (center[0] - 10, center[1] - 10), (center[0] + 10, center[1] - 10)])
    elif direction == "E":
        pygame.draw.polygon(screen, color, [(center[0] + 10, center[1]), (center[0] - 10, center[1] - 10), (center[0] - 10, center[1] + 10)])
    elif direction == "W":
        pygame.draw.polygon(screen, color, [(center[0] - 10, center[1]), (center[0] + 10, center[1] - 10), (center[0] + 10, center[1] + 10)])

#Drawing the way from the start to the end in red
def draw_path():
    x, y = 0, 0
    path_color = (255, 0, 0)
    visited = set()
    while (x, y) != (grid_size[0] - 1, grid_size[1] - 1):
        if (x, y) not in P or (x, y) in visited:
            break
        visited.add((x, y))
        arrow = P[(x, y)]
        next_x, next_y = x, y
        if arrow == "N":
            next_y += 1
        elif arrow == "S":
            next_y -= 1
        elif arrow == "E":
            next_x += 1
        elif arrow == "W":
            next_x -= 1
        
        #No infinite loops for the red arrows
        if (next_x, next_y) in visited:
            break
        
        draw_arrow(x, y, arrow, path_color)
        x, y = next_x, next_y

#Main
running = True
while running:
    screen.fill((200, 200, 200))
    draw_grid()
    draw_path()
    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()